# Grade Dashboard 

This Dashboard show course grades throughout a curriculum as tool for Higher Educators to see their dynamics of the curriculums. 

**These grades and assignments were computationally generated and are not real**

#explanatory 

Being able to import commonly used code is what make coding so powerful. In essence you can grab the knowledge of other experts and immediately reapply to your problem.

In [1]:
# Libraries for the data 
from collections import defaultdict #Make it is easier to create data strcuture for bar graph
import os #Talk to the operating system
import glob #Searches directories and files
import json #Reads javascript files
import numpy as np #Mathematical library
import pandas as pd #Dataframes -- python's Excel

# Dashboard Library
import solara # Key dashboard
import ipywidgets as widgets #Makes the dropdowns and sliders
# Plotting library
import plotly.express as px
import plotly.graph_objects as go

<IPython.core.display.Javascript object>

## 1 - Read in the course and assignment data
 
 Reads in the course list and assignments that were downloaded from Blackboard

#explanatory 

`with open` - opens the json (javascript object notation file that is located in this directory 

it then calls this data  *json_file* and uses the json library imported earlier to store it in memory with `json.load`

this data which contains all the course information is stored in the object drop_downs as it will create our drop down menus

#TOTRY
- Change `json_file` to any other name in both locations
- Add a cell with the plus button and write `drop_downs` this will show the json data structure of all the course material it may look confusing at first but with practice it becomes easy to read. 


In [2]:
with open('data/drop_downs.json', 'r') as json_file:
   drop_downs = json.load(json_file)

## 2 - Helper Functions 

This section is a list of functions we will repeatedly resue on the data

The helper functions are:
- `score_to_grade` - This function takes the raw number score and converts to a letter grade
- `final_grades` -  This creates a dataframe (table) that counts each of the letter grades so we can plot them on a bar graph
-  `make_overall` - Creates the bar graphs at the specified level form the drop_downs
-  `plot_grades` - Plots the individual grades
-  `grade_distribution` - Converts the information from the dashboard to the call the plotting functions

### Score to  Grade Function

Converts grades from numbers to letter

#explanatory

The function `score_to_grade` takes a number grade like 88.76 and converts into a letter grade so it can be plotted on the bar graph

The number grade is a parameter called `score` and the `>=` says if the number is greater than or equal to a number assign it a different letter grade

#TOTRY
- *Basic* - Change the letters or the numbers and watch the impact on the output
- *Advanced* - Change the distribution (e.g. add B+) For this to work you will also have to make change to `make_overall` function 

In [3]:
def score_to_grade(score):
    if score >= 93:
        return 'A'
    elif score >= 90:
        return 'A-'
    elif score >= 83:
        return 'B'
    elif score >= 80:
        return 'B-'
    elif score >= 78:
        return 'C+'
    elif score >= 70:
        return 'C'
    else:
        return 'F'

### Final Grade Function 

Create tables of letter grades to plot on bar graph

Calls the `score_to_grade` function 

#explanatory 

The function `final_grades` takes a course section (1 class) and an assignment, which are both strings (i.e. letters)

It creates an empty data structure called `grades` it then reads in the csv file (table) that has the grades. The code knows where to look based on the parameters section and assignment. 

If it is the final grade of the student (i.e. the grade they got in the section) then it converts the number grades to letter grades using the `score_to_grade` function we called earlier

This returns a list of grades (e.g. [A, B, A, B, C,..])

It then iterates through the list and counts the letter grades. This data is then stored in the grades data structure and just counts them. The grade data strcuture is then stored in a dataframe (table) and returned to `make_overall`

#TOTRY
- think of a course you want to call and then find it in the course_grades folder what are your `section` and `assignment` parameters 


In [28]:
def final_grades(section, assignment):
    grades = {}
    all_section = pd.read_csv(f"data/course_grades/{section}/{assignment}")
    if "final" in assignment: 
        grade_list = all_section['Final Grade'].apply(score_to_grade)
    else: 
        grade_list = all_section['Grades'].apply(score_to_grade)
    for g in grade_list: 
        if g not in grades.keys(): 
            grades[g] = 1
        else: 
            grades[g] += 1
    df = pd.DataFrame([grades])
    return df
                

### Make Overall Function 

Plots the Bar Graph 

#explanatory

This function plots the grades in a bar graph using the Plotly library we imported at the beginning. This comes with features like "mouse over" automatically. The `title` is based on inputs from the dropdown menus.

First, it cleans the data, it the faculty member entered in an unreadable column it changes the column name from unnamed to Unknown

Second, it accounts for all grades so say in a course did not give out a C+ it will add that column with a value of 0

Third, it ensures the columns are in the desired order of A to F

Finally, it plots the dataframe. The X axis is the column names (e.g. A, B ...) and the Y axis the count of those grades converted from the dataframe to a list of grades.

#TOTRY
- Change the title, X-axis or Y-axis of the graphs
- Change the layout size (e.g. `height=400`)


In [4]:
def make_overall(df, title): 
    # Get rid of unanmed 
    df.columns = [col if 'Unnamed' not in col else 'unknown' for col in df.columns]
    
    # Ensure all expected columns (A, B, C, D, F) are present, add missing ones with default value (e.g., None)
    expected_columns = ['A', 'A-','B','B-','C+', 'C','F', 'Unknown']
    for col in expected_columns:
        if col not in df.columns:
            df[col] = 0  
    
    # Reorder the columns in grade order
    df = df[expected_columns] 
    
    fig = go.Figure(data=[
    go.Bar(x=list(df.columns), y=df.iloc[0].tolist())  # x for categories, y for values
    ])
    # Customize the layout (optional)
    fig.update_layout(
        title=title,
        xaxis_title="Grades",
        yaxis_title="Frequency"
    )

    # Set the figure height to display all rows
    fig.update_layout(height=500)
    
    return solara.FigurePlotly(fig)  

### Plot Grade Function 

Horizontal Bar Graph plot of Sutdent ID (not real) and grade on Assignment

In [37]:
def plot_grades(program, course,section,assignment):
    dfs = []
    folders=[]
    file=f"data/course_grades\\{program}\\{course}\\{section}\\{assignment}.csv" 
    df = pd.read_csv(file)
    df = df.replace({pd.NA: None, np.nan: None})   
    df["Student ID"] = df['Student ID'].astype(str)
    fig = px.bar(df, 
             x='Grades', 
             y='Student ID', 
             orientation='h', 
             hover_data=["Feedback"],
             title=assignment)

    # Set the figure height to display all rows
    fig.update_layout(height=1000)
    
    # Set the x-axis range to start from the lowest grade minus 10
    min_grade = df['Grades'].min()
    max_grade = df["Grades"].max()
    fig.update_xaxes(range=[min_grade - 5, max_grade])  # Lower bound set, upper bound auto
 
    return solara.FigurePlotly(fig)

### Grade Distribution Function 

Takes the drop down menu inputs and calls:
- the `final_grades` function which converts the grades from numbers to letters and makes a table of the number
- the `make_overall` function which takes the table and plots a bar graph 


#explanatory

The grade distribution function parses the information form the drop down menu so that when it calls the `make_overall` function it is passing the right information to those functions. 

The first parameter tells what `level` of the drop down the user is working. If there has been no selection then the level is equal to the string "all" and it plots the overall grades for all courses. 

The second parameter is the `target`, this is specifically what the user is requesting (e.g. MSTI, BSI etc) this and updates the elvel to program

The third parameter is the `details` this is for when a user wants to to see the assignment grades and passes in the specific assignment of interest. 

These parameters then dictate what is call is made. The code looks reads in the data calling the `make_overall` with the dataframe (table) of grades and the title of the bar graph 

In [38]:
def grade_distribution(level, target, details):
    if level=="all":
        return make_overall(pd.read_csv("data/course_grades/overall_grades.csv"), "Overall Grade Distribution")
    if level=="program":
        #overall = get_data(details)
        return make_overall(pd.read_csv(f"data/course_grades/{target}/overall_grades.csv"), f"{target} Grade Distribution")
    if level=="course":
        try: #account for faculty who dont use blackboard
            return make_overall(pd.read_csv(f"data/course_grades/{target}/overall_grades.csv"), f"{target} Grade Distribution")
        except: 
            return solara.Markdown("## This course did not input grades in blackboard.")
    if level=="section": 
        section_grades = final_grades(target, "final_grade.csv")
        return make_overall(section_grades, f"{target} Grade Distribution")
    if level == "assignment":
        section_grades = final_grades(target,details)
        return make_overall(section_grades, f"Assignment Grade Distribution")

## 3 - Make the Dashboard

### Intialize the Interactive Widgets

Create the user input widgets in this case the drop down menus

#explanatory

This cell creates the interactive widgets which are the drop down menus and one button at the: 

Dropdowns: 
- program 
- course
- section
- assignment

Button:
- graphs of the scores by student ID (not real ids) 

In [39]:
# Reactive state management for dropdowns
selected_program = solara.reactive(None)
selected_course = solara.reactive(None)
selected_section = solara.reactive(None)
selected_assignments = solara.reactive(None)
graph_grades = solara.reactive(False) 
reset_selection =solara.reactive(False)

### Populate the Drop Down Menus

Fills the drop down menus with list of options based on user selection 

#explanatory

Each of these functions populates the drop down menus based on the initialization earlier.  So as users select a drop down it changes the value from `None` to whatever was selected. This then references the json file we read in earlier to populate the next drop down menu. 

For the plot grade status it is just a button that toggle from true to false and if true it plots the assignments by student ID (not real IDs) 

In [40]:
def reset_controls():
    selected_program.value = None
    selected_course.value = None
    selected_section.value = None
    selected_assignments.value = None
    reset_selection.value=False
            

def program_selected(program):
    if program != None: 
        courses = list(drop_downs[program].keys())
        courses.sort()
        return courses

def course_selected(course):
    if course != None: 
        if course not in list(drop_downs[selected_program.value].keys()):
            return None
        else: 
            sections = list(drop_downs[selected_program.value][course]["Sections"].keys())
            sections.sort()
            #plot
            return sections
    else: 
        return None

def section_selected(section):
    if section != None: 
        if section not in list(drop_downs[selected_program.value][selected_course.value]["Sections"].keys()):
            return None
        else: 
            assignments = drop_downs[selected_program.value][selected_course.value]["Sections"][section]
            assignments.sort()
            return assignments
    else: 
        return None

def plot_grade_status(): 
    graph_grades.set(not graph_grades.value)

### Output of Course Grades

Based on user selection generates the bar graphs of grade distribution 

#explanatory

This code controls what ie being displayed. It is wrapped in a `try` and `except` syntax. This prevents the code from erroring out, if say some selects MSTI >> MST613 but then goes back and changes to MSSI in which case there is not MST613. 

This code takes the inputs from the dropdown menu and then call the grade distirbution function.

The target variable is just a string (letters) to the path of the final that needs ot be plotted. 

In [47]:
@solara.component
def View():    
   
    try: 
        if selected_program.value == None: 
            grade_distribution("all", None, None)
        elif selected_program.value != None and selected_course.value == None:
            grade_distribution("program", selected_program.value, drop_downs[selected_program.value])
        elif selected_course.value != None and selected_section.value == None: 
            target =selected_program.value+"/"+selected_course.value
            grade_distribution("course", target, drop_downs[selected_program.value][selected_course.value])
        elif selected_section.value != None and selected_assignments.value == None: 
            target =selected_program.value+"/"+selected_course.value+"/"+selected_section.value
            grade_distribution("section", target, None) 
        elif selected_assignments != None:  
            target = selected_program.value+"/"+selected_course.value+"/"+selected_section.value
            assignment = selected_assignments.value +".csv"
            grade_distribution("assignment", target,assignment)
            solara.Button(label="Show Grades by Student ID", on_click=plot_grade_status)
            if graph_grades.value== True: 
                plot_grades(selected_program.value, selected_course.value, selected_section.value, selected_assignments.value) 
        
    except: 
        solara.Markdown("## Selections do not compute, please update selections")
        reset_controls()

### Drop Down Menus

Displays the drop down menus

#explanatory 

This cell creates the dropdowns we iniatilized earlier in this case we are using `Select` which allows users to select one value form the menu. 

This takes three parameters 
 - label of the dropdown
 - value of the dropdown (they are inialized with `None` at the beginning of section 3
 - values to populate the dropdown

This is also wrapped in a `try` and `except` to prevent errors for mismatched selections

#TOTRY
- Change the error message you get with a mimatched selection

In [48]:
@solara.component
def Controls():     
    solara.Button(label="Reset Selection", on_click=reset_controls)
    try: 
        solara.Select(label="Program", value=selected_program, values=list(drop_downs.keys()))
        solara.Select(label="Course", value=selected_course, values = program_selected(selected_program.value))
        #TODO change to SelectMultiple 
        solara.Select(label="Section", value=selected_section, values = course_selected(selected_course.value))
        #TODO change to SelectMultiple
        solara.Select(label="Assignments", value = selected_assignments, values = section_selected(selected_section.value))
    except: 
        solara.Markdown("## Selections do not compute, please update selections")

### Title of the Dashboard

Creates the title of the dashboard

#explanatory

This cell just provides the title of the dashboard

#TOTRY
- Change the title
- Change the number of hashtags

In [49]:
@solara.component
def Title():
        solara.Markdown("## Assignment Grade Dashboard")

### Create the Dashboard

 - Calls the display area `View` function
 - Calls the drop down menus `Control` function
 - Calls the `Title` function 
 

#explanatory

This code brings it all together to make the dashboard. 

It calls the `Title`, `Controls` (placing them in the sidebar) and `View` which puts together all the pieces of the dashboard. 

In [50]:
@solara.component
def Page():
    Title()
    with solara.Sidebar():
        Controls()
    View()
     

Page()

Cannot show ipywidgets in text